cell 1

In [1]:
!pip install --quiet --no-deps together==1.5.13
!sudo apt install tesseract-ocr
!pip install pip==23.3.1
!pip install textract pytesseract pillow pandas matplotlib together openpyxl python-docx

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


cell 2

In [2]:
import os
import pandas as pd
import numpy as np
import base64
import textract
import matplotlib.pyplot as plt
from IPython.display import display, Image
import together
import re
from io import BytesIO
from PIL import Image as PILImage
import pytesseract

cell 3

In [3]:
os.environ["TOGETHER_API_KEY"] = "tgp_v1_6K5twiAUdIv6ULMjEZOlw_zlabob2DsUBYKgb5Kaz_I" #Hi this is prakarsh and its a API key
together.api_key = os.environ["TOGETHER_API_KEY"]


cell 4

In [4]:
def read_file(file_path):
    ext = os.path.splitext(file_path)[-1].lower()

    if ext in [".csv", ".xls", ".xlsx"]:
        return pd.read_csv(file_path) if ext == ".csv" else pd.read_excel(file_path)
    elif ext in [".txt", ".pdf", ".docx"]:
        return textract.process(file_path).decode("utf-8")
    elif ext in [".png", ".jpg", ".jpeg"]:
        return pytesseract.image_to_string(PILImage.open(file_path))
    else:
        raise ValueError(f"Unsupported file format: {ext}")


cell 5

In [5]:
def optimize_context(data, max_chars=3000):
    if isinstance(data, pd.DataFrame):
        summary = f"DataFrame: {len(data)} rows × {len(data.columns)} columns\n"
        summary += f"Columns: {', '.join(data.columns)}\n"
        sample_size = min(3, len(data))
        summary += f"\nSample data (first {sample_size} rows):\n"
        summary += data.head(sample_size).to_string(index=False)
        num_cols = data.select_dtypes(include=np.number).columns
        if not num_cols.empty:
            summary += "\n\nNumerical columns statistics:\n"
            stats = data[num_cols].describe().loc[['mean', 'min', 'max']]
            summary += stats.to_string()
        return summary[:max_chars]

    elif isinstance(data, str):
        return data[:max_chars//2] + "\n\n...TRUNCATED...\n\n" + data[-max_chars//2:] if len(data) > max_chars else data
    return str(data)[:max_chars]

cell 6

In [6]:
def generate_response(question, context):
    """Generate response with context"""
    prompt = f"""
[CONTEXT]
{context}

[QUESTION]
{question}

[RESPONSE]
"""
    try:
        response = together.Complete.create(
            model="meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8",
            prompt=prompt,
            max_tokens=1024,
            temperature=0.7,
            stop=["</s>", "[QUESTION]"]
        )
        # FIX: Handle new response format
        if 'output' in response and 'choices' in response['output']:
            return response['output']['choices'][0]['text'].strip()
        elif 'choices' in response:
            return response['choices'][0]['text'].strip()
        else:
            return "Error: Unexpected API response format"
    except Exception as e:
        return f"API Error: {str(e)}"
def extract_code(response_text):
    """Extract Python code from model response"""
    code_blocks = re.findall(r"```python(.*?)```", response_text, re.DOTALL)
    return code_blocks[0].strip() if code_blocks else ""

def execute_plot_code(code, df):
    """Execute code and return base64 image"""
    local_vars = {"df": df, "plt": plt}
    plt.figure()  # Create new figure

    try:
        exec(code, {}, local_vars)
        buf = BytesIO()
        plt.savefig(buf, format="png", bbox_inches="tight")
        plt.close()
        return base64.b64encode(buf.getvalue()).decode("utf-8")
    except Exception as e:
        return f"Error: {str(e)}"

cell 7

In [7]:
class DataAnalystAgent:
    def __init__(self):
        self.data = None
        self.data_type = None
        self.context_summary = ""

    def ingest_file(self, file_path):
        try:
            self.data = read_file(file_path)
            self.data_type = 'dataframe' if isinstance(self.data, pd.DataFrame) else 'text'
            self.context_summary = optimize_context(self.data)
            return f"File ingested. Data type: {self.data_type}"
        except Exception as e:
            return f"Error processing file: {str(e)}"
    def ask_question(self, question):
        if self.data is None:
            return "Please upload a file first", None
        try:
            response = generate_response(question, self.context_summary)
            img_b64 = None

            # Check if visualization is requested
            if "plot" in question.lower() or "visualization" in question.lower() or "chart" in question.lower():
                code = extract_code(response)
                if code:
                    img_b64 = execute_plot_code(code, self.data)
            return response, img_b64
        except Exception as e:
            return f"Error: {str(e)}", None

cell 8

In [14]:
from google.colab import files

uploaded = files.upload()
file_path = next(iter(uploaded))
agent = DataAnalystAgent()
print(agent.ingest_file(file_path))

Saving Screenshot 2025-06-09 180927.png to Screenshot 2025-06-09 180927.png
File ingested. Data type: text


cell 9

In [15]:
if agent.data_type == 'dataframe':
    display(agent.data.head())
else:
    print(agent.context_summary[:500])

+ LLM-Based Auto Slide Deck Creator with Web Data Integration
Built a Python tool to auto-generate PowerPoint decks using LLMs and real-time web search.

— Tools & technologies used: VS Code, Python, OpenRouter API (LLM) SerpAPI (Google Search), python-pptx,
REST APIs, dotenv, requests

— Developed a Python-based tool that auto-generates structured PowerPoint presentations on any topic by integrat-
ing LLMs (via OpenRouter) and real-time data from Google Search (via SerpAPI). Utilized REST APIs 


cell 10

In [16]:
question = input("Ask your question about the data: ")

try:
    response, img_b64 = agent.ask_question(question)
    print("\nRESPONSE:\n", response)

    if img_b64 and not img_b64.startswith("Error"):
        display(Image(base64.b64decode(img_b64)))
    elif img_b64:
        print(img_b64)
except Exception as e:
    print("Error:", e)

Ask your question about the data: summarise me what this is


<ipython-input-6-662108674>:13: DeprecationWarning: Call to deprecated function create.
  response = together.Complete.create(
/usr/local/lib/python3.11/dist-packages/together/legacy/complete.py:23: UserWarning: The use of together.api_key is deprecated and will be removed in the next major release. Please set the TOGETHER_API_KEY environment variable instead.
  warnings.warn(API_KEY_WARNING)



RESPONSE:
 This appears to be a summary of a person's technical skills, experience, and interests, likely from a resume or a personal profile. It highlights their:

1. Projects: 
   - Auto Slide Deck Creator using LLMs and web data integration
   - Autism Prediction using Machine Learning

2. Technical Skills: 
   - Programming languages (C, C++, Python, HTML, CSS)
   - Developer tools (Jupyter Notebook, Google Colab, Git/GitHub, etc.)
   - Frameworks (NumPy, Pandas, Matplotlib, Scikit-learn, etc.)

3. Soft Skills: 
   - Problem-solving, Critical Thinking, Team Collaboration, Adaptive Thinking

4. Coursework and Areas of Interest: 
   - Relevant coursework (Data Structures and Algorithms, Linear Algebra, etc.)
   - Interests in AI, ML, Computer Vision, Data Science, Physics, and more

5. Positions of Responsibility: 
   - Roles held in various clubs and cells, such as Innovation Cell and MotoSports Club at NIT.
